<img src="http://stech2.firstpost.com/tech2images/640x359/proportional/jpeg/2016/07/Pokemon-GO-Tech2-720-624x351.jpg" style="width:450px;height:220px;">

# Pokemon Project - CST495
By: Monique La Croix & Sonali Bharat

Here we work with a data set from Kaggle. We wanted to create a search engine that used characters from Pokemon. The data contains name, type1, and type2. We still ended up with some issues regarding the Inverted Index, TF-IDF, and minor other ones. With more time, we would have changed the database where it has image URL to Pokemon description.

First we load the data file and _normalise_ the text - removing certain characters and converting to lower case.  


In [7]:
import csv
import re

with open("data/pokemonData.txt") as f:
    r = csv.reader(f, delimiter=',', quotechar='"')
    rgx = re.compile(r'\b[a-zA-Z]+\b') 
    docs = [ (' '.join(re.findall(rgx, x[0])).lower(), ' '.join(re.findall(rgx, x[1])).lower(), ' '.join(re.findall(rgx, x[2])).lower(), 
             ' '.join(re.findall(rgx, x[3])).lower(), ' '.join(re.findall(rgx, x[4])).lower(), ' '.join(re.findall(rgx, x[5])).lower(),
             ' '.join(re.findall(rgx, x[5])).lower())  
            for i,x in enumerate(r) if i > 1 ]
# //Pokemon No.,Name,Type 1,Type 2,Max CP,Max HP,Image URL   
print(docs[0][1], docs[0][2], docs[0][3], docs[0][4])

# items_t = [ d[0] for d in docs ] # num

items_d = [ [d][1] for d in docs ] # name
items_t2 = [ d[2] for d in docs ] #type1
items_t3 = [ d[3] for d in docs ] #type2
items_t4 = [ d[4] for d in docs ] #max CP
# items_t5 = [ d[5] for d in docs ] #max HP
# items_t6 = [ d[6] for d in docs ] #image URL

# print ("test")

items_i = range(0, len(items_t2)) # num


('ivysaur', 'grass', 'poison', '')


## Term Frequency


In [101]:
corpus = items_t2[0:151]
print(corpus)

['grass', 'grass', 'fire', 'fire', 'fire', 'water', 'water', 'water', 'bug', 'bug', 'bug', 'bug', 'bug', 'bug', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'normal', 'poison', 'poison', 'electric', 'electric', 'ground', 'ground', 'poison', 'poison', 'poison', 'poison', 'poison', 'poison', 'fairy', 'fairy', 'fire', 'fire', 'normal', 'normal', 'poison', 'poison', 'grass', 'grass', 'grass', 'bug', 'bug', 'bug', 'bug', 'ground', 'ground', 'normal', 'normal', 'water', 'water', 'fighting', 'fighting', 'fire', 'fire', 'water', 'water', 'water', 'psychic', 'psychic', 'psychic', 'fighting', 'fighting', 'fighting', 'grass', 'grass', 'grass', 'water', 'water', 'rock', 'rock', 'rock', 'fire', 'fire', 'water', 'water', 'electric', 'electric', 'normal', 'normal', 'normal', 'water', 'water', 'poison', 'poison', 'water', 'water', 'ghost', 'ghost', 'ghost', 'rock', 'psychic', 'psychic', 'water', 'water', 'electric', 'electric', 'grass', 'grass', 'ground', 'ground', 'fighting', 'fighting

We started by computing the frequency of terms in the *entire* corpus. We built a python dictionary where the key is the token and the value is the count.

Here is part of where we computed term frequency.  

In [148]:
tf = {}
for doc in corpus:
    for word in doc.split():
        if word in tf:
            tf[word] += 1
        else:
            tf[word] = 1        

print(tf)

{'ghost': 3, 'ice': 2, 'electric': 9, 'normal': 22, 'fire': 12, 'psychic': 8, 'poison': 14, 'dragon': 3, 'water': 28, 'fighting': 7, 'rock': 9, 'fairy': 2, 'grass': 11, 'bug': 12, 'ground': 8}


This function definition computes term frequency using the `Counter`.

In [147]:
from collections import Counter

def get_tf(corpus):
    tf = Counter()
    for doc in corpus:
        for word in doc.split():
            if word in tf:
                tf[word] = tf[word] + 1
            else:
                tf[word] = 1
            
            
    return tf

tf = get_tf(corpus)
print(tf)

Counter({'water': 28, 'normal': 22, 'poison': 14, 'fire': 12, 'bug': 12, 'grass': 11, 'electric': 9, 'rock': 9, 'psychic': 8, 'ground': 8, 'fighting': 7, 'ghost': 3, 'dragon': 3, 'ice': 2, 'fairy': 2})


 
We used the Jupyter Notebook [magics](http://nbviewer.jupyter.org/github/ipython/ipython/blob/1.x/examples/notebooks/Cell%20Magics.ipynb) to time the execution.

In [44]:
%%timeit 
tf = get_tf(corpus)

The slowest run took 5.40 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 534 µs per loop


We used the function `get_tf()` to compute term frequency for the across the large corpus of type1 in the list `items_t`.      
We found out the frequency of the term 'Grass'.

In [48]:
## HIDE
tf = get_tf(items_t2)
print(tf['Grass'])
## HIDE

0


## Document Term Frequency

<img src="http://shirtigo.co/wp-content/uploads/2014/01/fourelements.jpg" style="width:390px;height:220px;">

So far we have computed the term frequency across the entire corpus.  The term frequency could also be computed for each document. 

We stored the document term frequency in a dictionary, where the key is the document id and the value is the a nested dictionary of document terms and their counts.

For example consider this corpus:

1. 'Grass'
2. 'Fire'
3. 'Water'
4. 'Bug'
5. 'Normal' 
6. 'Poison'
7. 'Electric'
8. 'Ground'
9. 'Fairy'
10. 'Fighting'
11. 'Psychic'
12. 'Rock'
13. 'Ghost'
14. 'Ice'
15. 'Dragon'

We completed the function definintion `get_tf()` to compute the **document term frequencies** for a single document.  This function is then called once per document in the corpus by the function `get_df()`.  

In [149]:
import collections

def get_tf(document):
    tf = Counter()
    ## HIDE
    for word in document.split():
        tf[word] += 1
    ## HIDE
    return tf

def get_dtf(corpus):
    dtf = {}
    for i,doc in enumerate(corpus):
        dtf[i]=get_tf(doc)
    return dtf
            
    
dtf = get_dtf(items_t2)
dtf[4]

Counter({'fire': 1})

<img src="https://boost-rankedboost.netdna-ssl.com/wp-content/uploads/2016/07/PokeBall.png" style="float:right;width:140px;height:140px;">

## Term Frequency Matrix

Whilst the document term frequency dictionary in the previous section `tfd` is a compact way to store the term frequency it is not efficient for analysis.  A term frequency matrix is a more effective way to store the data if we wish to perform analysis - such as compare or rank documents. 

For example consider this corpus:

1. 'Grass'
2. 'Fire'
3. 'Water'
4. 'Bug'
5. 'Normal' 
6. 'Poison'
7. 'Electric'
8. 'Ground'
9. 'Fairy'
10. 'Fighting'
11. 'Psychic'
12. 'Rock'
13. 'Ghost'
14. 'Ice'
15. 'Dragon'

There is a total vocabulary of 15 terms.

Those arrays can be stacked naturally into a matrix - one row per document, one column per term.  We call this matrix the term frequency matrix.

To compute the term frequency matrix we have to first compute the lexicon (set of terms) in our corpus.

In [146]:
s = set()
s.add('Grass')
s.add('Fire')
s.add('Water')
s.add('Bug')
s.add('Normal')
s.add('Poison')
s.add('Electric')
s.add('Ground')
s.add('Fairy')
s.add('Fighting')
s.add('Psychic')
s.add('Rock')
s.add('Ghost')
s.add('Ice')
s.add('Dragon')
print s

set(['Ghost', 'Electric', 'Ice', 'Normal', 'Fire', 'Psychic', 'Poison', 'Dragon', 'Water', 'Fighting', 'Rock', 'Fairy', 'Grass', 'Bug', 'Ground'])


Here is the `get_lexicon()` function definition so that it returns a list of unique terms for a given corpus of documents.

In [150]:
def get_lexicon(corpus):
    lexicon = set()
   
    for doc in corpus:
            lexicon.update(word for word in doc.split())
    
    return list(lexicon)
    
test_corpus = ['Grass','Rock','Psychic']
lexicon = get_lexicon(test_corpus)
print lexicon

['Psychic', 'Grass', 'Rock']


Next we computed a term frequency matrix - the term frequency matrix has one row per document in our corpus and one column per term in our lexicon.   

The function definition below computes the term frequency matrix for a corpus of documents. 

In [151]:
def get_tfm(corpus):
    
    def get_lexicon(corpus):
        lexicon = set()
        for doc in corpus:
            lexicon.update([word for word in doc.split()])
        return list(lexicon)
        
    lexicon = get_lexicon(corpus)

    tfm =[]
    for doc in corpus:
        tfv = [0]*len(lexicon)
        for term in doc.split():
            tfv[lexicon.index(term)] += 1
            
        tfm.append(tfv)
        
    return tfm, lexicon


test_corpus = ['Grass','Rock','Psychic']
tfm, lexicon = get_tfm(test_corpus)
print(lexicon)
print(tfm)

['Psychic', 'Grass', 'Rock']
[[0, 1, 0], [0, 0, 1], [1, 0, 0]]


### Sparsity of term frequency matrix

As our corpus increases so does the sparsity of the term frequency matrix - most elements have value zero.  
We used more efficient [sparse matrix](https://en.wikipedia.org/wiki/Sparse_matrix) storage to save memory.

In [59]:
import pandas as pd
from bokeh.plotting import figure, output_notebook, show, vplot

# Sparsity as a function of document count
n = []
s = []
for i in range(100,1000,100):
    corpus = items_t[0:i]
    tfm, lexicon = get_tfm(corpus)
    c =[ [x.count(0), x.count(1)] for x in tfm]
    n_zero = sum([ y[0] for y in c])
    n_one = sum([ y[1] for y in c])  
    s.append(1.0 - (float(n_one) / (n_one + n_zero)))
    n.append(i)
    
output_notebook(hide_banner=True)
p = figure(x_axis_label='Documents', y_axis_label='Sparsity',
          plot_width=400, plot_height=400)
p.line(n, s, line_width=2)
p.circle(n, s, fill_color="white", size=8)
show(p)

<img src="https://ih0.redbubble.net/image.89981518.4942/flat,800x800,075,t.u2.jpg" style="float:right;width:250px;height:250px;">

## Boolean Search

This is our first ranking function. Now we have the term frequency matrix, which we used to find documents that contain words included in a user specified query. We started by simply returning the documents from the corpus that matched any terms in the query and ranked by the raw frequency of matching terms. 

More specifically our algorithm for 'boolean search' proceeds as follows:

* Compute the lexicon for the corpus
* Compute the term frequency matrix for the corpus
* Convert query to query vector using the same lexicon 
* Compare each documents term frequncy vector to the query vector - specifically for each document in the corpus:
  * Compute a ranking score for each document by taking the [dot product](https://en.wikipedia.org/wiki/Dot_product) of the document's term frequency vector and the query vector
* Sort the documents by ranking score

In [8]:
test_corpus = ['Grass','Rock','Psychic']
print(test_corpus)

# Compute term frequency matrix and lexicon
tfm, lexicon = get_tfm(test_corpus)

print(tfm)
print(lexicon)

# Define our query
qry = 'red bike'

# Convert query to query vector using lexicon
qrv = [0]*len(lexicon)
for term in qry.split():
    if term in lexicon:
        qrv[lexicon.index(term)] = 1
            
print(qrv)

# Compare query vector to each term frequency vector
# This is dot product between qrv and each row of tfm
for i,tfv in enumerate(tfm):
    print i , sum([ xy[0] * xy[1] for xy in zip(qrv,tfv)])
    

['Grass', 'Rock', 'Psychic']


NameError: name 'get_tfm' is not defined

The function definition `get_results_tf()` computes the document ranking score for each document in the term frequency matrix. The function returns a list of document id's sorted by their score. Test using a Pokemon related query such as 'Fire'. 

In [152]:
def get_results_tf(qry, tfm, lexicon):
    qrv = [0]*len(lexicon)
    for term in qry.split():
        if term in lexicon:
            qrv[lexicon.index(term)] = 1

    results = []      
    for i, tfv in enumerate(tfm):
        score = 0
        score = sum([ xy[0] * xy[1] for xy in zip(qrv,tfv)])
        
        results.append([score, i])
    
    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results


def print_results(results,n, head=True):
    ''' Helper function to print results
    '''
    if head:    
        print('\nTop %d from recall set of %d items:' % (n,len(results)))
        for r in results[:n]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    else:
        print('\nBottom %d from recall set of %d items:' % (n,len(results)))
        for r in results[-n:]:
            print('\t%0.2f - %s'%(r[0],items_t[r[1]]))
    

tfm, lexicon = get_tfm(items_t)
results = get_results_tf('Fire', tfm , lexicon)
print_results(results,10)



Top 10 from recall set of 150 items:
	0.00 - ivysaur
	0.00 - venusaur
	0.00 - charmander
	0.00 - charmeleon
	0.00 - charizard
	0.00 - squirtle
	0.00 - wartortle
	0.00 - blastoise
	0.00 - caterpie
	0.00 - metapod


## Inverted Index

We created an inverted index. The inverted index is used to filter out documents that do not contain any of the keywords in the query before computing the ranking score.  We know that these documents would score zero in the ranking scheme we are implementing.

Using our example mini-corpus:

1. 'Grass'
2. 'Rock'
3. 'Psychic'

We stored our inverted index in a dictionary where the key is the term and the value is the list of matching document id's.

We created an inverted index as a python dictionary keyed on the token, and validated with corpus. 

In [65]:
def create_inverted_index(corpus):
    idx={}
    for i, document in enumerate(corpus):
           for word in document.split():
            if word in idx:
                idx[word].append(i)
            else:
                idx[word] = [i]
        
    return idx

test_corpus = ['Grass','Rock','Psychic']
idx = create_inverted_index(test_corpus)
print(idx)

{'Psychic': [2], 'Grass': [0], 'Rock': [1]}


Here we created an inverted index for all the type2s.  We can use the set intersection method to find all the documents that match the query.

In [108]:
idx = create_inverted_index(items_t2)
print(set(idx['fire']))
print(items_t2[144])

set([2, 35, 4, 134, 75, 76, 144, 3, 56, 57, 124, 36])
fire


Here we're only scoring the documents that match our keywords in the query:

In [116]:
def get_results_tf(qry, idx):
    score = Counter()
    for term in qry.split():
        for doc in idx[term]:
            score[doc] += 1
            
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results;


idx = create_inverted_index(items_t2)
results = get_results_tf('fire', idx)
print_results(results,151)


Top 151 from recall set of 12 items:
	1.00 - charmander
	1.00 - vulpix
	1.00 - charizard
	1.00 - flareon
	1.00 - ponyta
	1.00 - rapidash
	1.00 - moltres
	1.00 - charmeleon
	1.00 - growlithe
	1.00 - arcanine
	1.00 - magmar
	1.00 - ninetales


The term frequency ranking is dominated by high frequency terms. 

In [9]:
# import pandas as pd
# from bokeh.plotting import output_notebook, show
# from bokeh.charts import Bar
# from bokeh.charts.attributes import CatAttr
# #from bokeh.models import ColumnDataSource

# df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()]})

# output_notebook(hide_banner=True)
# p = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
#         plot_width=800, plot_height=400)
# show(p)


## TF-IDF

We have all the information we need to compute IDF.  Reminder the Inverse Document Frequency is a weight we apply to each term in our lexicon.  The 'smooth' IDF for a specific term is computed as follows:

$$
IDF = log ( 1 + \frac{N}{n_t} ) 
$$

Where $N$ is the number of documents in the corpus and $n_t$ is the number of documents in the corpus in which the term $t$ appears.  $n_t$ is simply the length of the list of documents for a given key in our inverted index.

This function definition computes IDF for a given term, given an inverted index `idx` and number of documents `n`. The log function is in the [math](https://docs.python.org/2/library/math.html) module. 

In [114]:
import math

def idf(term, idx, n):
    return math.log( float(n) / (1 + len(idx[term])))

     
print(idf('fire',idx,len(items_t2)))
print(idf('ice',idx,len(items_t2)))


2.44568593663
3.91202300543


<img src="https://embed.gyazo.com/7028e5c1f631da5b2cbebdb9a0685214.png" style="float:left;width:260px;height:285px;">

### TF-IDF Intuition

We plotted the relationship between TF and IDF and got more intuition for what TF and IDF is all about. 

In [86]:
from bokeh.charts import vplot

idx = create_inverted_index(items_t)

df = pd.DataFrame({'term':[x for x in idx.keys()],'freq':[len(x) for x in idx.values()],
                  'idf':[idf(x, idx, len(items_t)) for x in idx.keys()]})

output_notebook(hide_banner=True)
p1 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='freq',
        plot_width=800, plot_height=400)
p2 = Bar(df.sort_values('freq', ascending=False)[:30], label=CatAttr(columns=['term'], sort=False), values='idf',
        plot_width=800, plot_height=400)
p = vplot(p1, p2)
show(p)

### TD-IDF Ranking 

After, we ranked based on TF-IDF which involved `get_results_tf()`, but we changed:

1. We need to know how many times the term `t` appears in `D`.  We can store this in our inverted index.  Instead of storing the document ID we can add the document ID and the number of times the term appears.  Previously this was captured in the TF matrix. We can avoid computing the TF matrix if adjust our index.
2. We have to change the function signature in the ranking function - passing in the total size of the corpus.
3. We also have to change the score calculation to incorporate the IDF weighting.

The `get_results_tfidf()` function scores documents based on TF-IDF.  

In [118]:
def create_inverted_index(corpus):
    idx={}
    for i, doc in enumerate(corpus):
        for word in doc.split():
            if word in idx:
                if i in idx[word]:
                    # Update document's frequency
                    idx[word][i] += 1
                else:
                    # Add document
                    idx[word][i] = 1
            else:
                # Add term
                idx[word] = {i:1}
    return idx

def get_results_tfidf(qry, idx, n):
    score = Counter()
    for term in qry.split():
        # << IMPLEMENT TF-IDF SCORING >> 
        # CODE HERE >>
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                score[doc] += idx[term][doc] * i
        
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])
    
    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

idx = create_inverted_index(items_t2)
results = get_results_tfidf('rock', idx, len(items_t2))
print_results(results,151)


Top 151 from recall set of 9 items:
	2.71 - geodude
	2.71 - graveler
	2.71 - golem
	2.71 - kabutops
	2.71 - aerodactyl
	2.71 - onix
	2.71 - omanyte
	2.71 - omastar
	2.71 - kabuto


### Problematic Queries

Although we have 'fixed' the suspension query, there was another problem which seemed to just return a heap of accessories:

In [123]:
idx = create_inverted_index(items_t2)
results = get_results_tfidf('ice', idx, len(items_t2))
print_results(results,151)


Top 151 from recall set of 2 items:
	3.91 - jynx
	3.91 - articuno


The items scoring highly have more terms in them.  The more terms in a title the more likely it is to match. 

This would lead us to hypothesize that we should penalise items where there are many more terms in the item title.  We might assume that results that score highly but have fewer terms are better that results that score highly but contain many terms.   

In addition TF-IDF seems to result in many items getting exactly the same score - we would prefer to be able to seperate out the results rather than have 8 scoring 4.06. 

In [127]:
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'title':[items_t[x[1]] for x in results]})

d = df.groupby('score').first().reset_index()

r1 = re.compile('(Fire)')
r2 = re.compile('(Ice)')

for i, t in enumerate(d.title):
    n1 = r1.findall(t)
    n2 = r2.findall(t)
    print('%d x Fire, %d x Ice, Score = %0.2f'%(len(n1),len(n2),d.score[i]))

0 x Fire, 0 x Ice, Score = 3.91


In [10]:
# from bokeh.plotting import output_notebook, show
# from bokeh.charts import Scatter

# # Plot score vs item length
# df = pd.DataFrame({'score':[float(x[0]) for x in results],
#                    'length':[len(items_t2[x[1]].split()) for x in results]})

# output_notebook(hide_banner=True)
# p = Scatter(df, x='score', y='length')
# show(p)

We did not want scores to be the same for lots of documents. High TF-IDF scores in shorter documents should be more relevant - so we boosted the score for documents that are shorter than average.

In [134]:
def get_results_tfidf_boost(qry, corpus):
    idx = create_inverted_index(corpus)
    n = len(corpus)
    d = [len(x.split()) for x in corpus]
    d_avg = float(sum(d)) / len(d)
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                f = float(idx[term][doc])
                score[doc] += i *  ( f / (float(d[doc]) / d_avg) )
        
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            # output [0] score, [1] doc_id
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

In [137]:
results = get_results_tfidf_boost('Grass', items_t2)
print_results(results, 151)

# Plot score vs item length
df = pd.DataFrame({'score':[float(x[0]) for x in results],
                   'length':[len(items_t2[x[1]].split()) for x in results]})

output_notebook()
p = Scatter(df, x='score', y='length')
show(p)


Top 151 from recall set of 0 items:


BokehJS successfully loaded.

ValueError: Out of range float values are not JSON compliant: nan

## Implementing BM25

Here is where we implemented the BM25 algorithm.    

In [138]:
def get_results_bm25(qry, corpus, k1=1.5, b=0.75):
    idx = create_inverted_index(corpus)
    # 1.Assign (integer) n to be the number of documents in the corpus
    ## HIDE
    n = len(corpus)
    # 2.Assign (list) d with elements corresponding to the number of terms in each document in the corpus
    ## HIDE
    d = [len(x.split()) for x in corpus]
    # 3.Assign (float) d_avg as the average document length of the documents in the corpus
    ## HIDE
    d_avg = float(sum(d)) / len(d)                
    score = Counter()
    for term in qry.split():
        if term in idx:
            i = idf(term, idx, n)
            for doc in idx[term]:
                # 4.Assign (float) f equal to the number of times the term appears in doc
                ## HIDE
                f = float(idx[term][doc])
                # 5.Assign (float) s the BM25 score for this (term, document) pair
                # HIDE
                s = i * (( f * (k1 + 1) ) / (f + k1 * (1 - b + (b * (float(d[doc]) / d_avg)))))
                score[doc] += s
                
    results=[]
    for x in [[r[0],r[1]] for r in zip(score.keys(), score.values())]:
        if x[1] > 0:
            results.append([x[1],x[0]])

    sorted_results = sorted(results, key=lambda t: t[0] * -1 )
    return sorted_results

In [140]:
results = get_results_bm25('water', items_t2)
print_results(results, 151)


Top 151 from recall set of 28 items:
	1.64 - gyarados
	1.64 - lapras
	1.64 - vaporeon
	1.64 - squirtle
	1.64 - wartortle
	1.64 - blastoise
	1.64 - psyduck
	1.64 - golduck
	1.64 - poliwag
	1.64 - poliwhirl
	1.64 - poliwrath
	1.64 - tentacool
	1.64 - tentacruel
	1.64 - slowpoke
	1.64 - slowbro
	1.64 - seel
	1.64 - dewgong
	1.64 - shellder
	1.64 - cloyster
	1.64 - krabby
	1.64 - kingler
	1.64 - horsea
	1.64 - seadra
	1.64 - goldeen
	1.64 - seaking
	1.64 - staryu
	1.64 - starmie
	1.64 - magikarp


Here we tested it with different parameters.

In [11]:
# results = get_results_bm25('water', items_t2, k1=1.5, b=0.75)

# # Plot score vs item length
# df = pd.DataFrame({'score':[float(x[0]) for x in results],
#                    'length':[len(items_t2[x[1]].split()) for x in results]})
# output_notebook()
# p = Scatter(df, x='score', y='length')
# show(p)